In [1]:
import pandas as pd

In [2]:
df1 = pd.read_parquet(
    "/home/pl487/allset/outputs/dry_run/agnews_bert_tiny_2023-03-20T16-26-23/labelled_dataset.parquet"
)
df2 = pd.read_parquet(
    "/home/pl487/allset/outputs/dry_run/agnews_bert_tiny_2023-03-20T16-28-45/labelled_dataset.parquet"
)

In [3]:
df1.equals(df2)

True

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from copy import deepcopy
from pathlib import Path

import pandas as pd
import seaborn as sns
import srsly
from datasets import load_from_disk
from lightning.fabric.loggers import TensorBoardLogger
from scipy.special import entr, softmax
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from src.data import ClassificationActiveDataModule, ClassificationDataModule
from src.energizer.utilities import local_seed
from src.estimators import (  # UncertaintyBasedStrategyForSequenceClassification,; SEALSRandomStrategyForSequenceClassification,
    EstimatorForSequenceClassification,
    RandomStrategyForSequenceClassification,
)

pd.set_option("display.max_colwidth", None)

In [3]:
data_path = Path("../data/prepared/agnews_bert_tiny/")

In [4]:
metadata = srsly.read_yaml(data_path / "metadata.yaml")
tokenizer = AutoTokenizer.from_pretrained(metadata["name_or_path"])

---
### Train

In [ ]:
dataset_dict = load_from_disk(data_path)
train_val = dataset_dict["train"].train_test_split(0.3)
dataset_dict["train"] = train_val["train"]
dataset_dict["validation"] = train_val["test"]

datamodule = ClassificationDataModule.from_dataset_dict(
    dataset_dict, tokenizer=tokenizer, batch_size=64, eval_batch_size=256
)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    metadata["name_or_path"],
    id2label=datamodule.id2label,
    label2id=datamodule.label2id,
    num_labels=len(datamodule.labels),
)

In [ ]:
estimator = EstimatorForSequenceClassification(model, accelerator="cuda")

In [ ]:
out = estimator.fit(
    train_loader=datamodule.train_loader(),
    validation_loader=datamodule.validation_loader(),
    limit_train_batches=10,
    limit_validation_batches=30,
    validation_interval=1,
)

In [ ]:
test_out = estimator.test(
    datamodule.test_loader(),
    limit_batches=11,
    # progress_bar=False,
)

---
### Active train

In [6]:
dataset_dict = load_from_disk(data_path)
datamodule = ClassificationDataModule.from_dataset_dict(
    dataset_dict, tokenizer=tokenizer, batch_size=64, eval_batch_size=256
)
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['unique_id', 'labels', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['unique_id', 'labels', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7600
    })
})

In [28]:
active_datamodule = ClassificationActiveDataModule.from_dataset_dict(
    dataset_dict, tokenizer, seed=42, batch_size=64, eval_batch_size=256
)

In [35]:
df = pd.read_parquet(
    "/home/pl487/allset/outputs/dry_run/agnews_bert_tiny_2023-03-20T18-10-13/labelled_dataset.parquet"
)

In [36]:
df["is_labelled"].sum()

125

In [30]:
active_datamodule.set_labelled_dataset(df)

In [34]:
active_datamodule.train_size(1)

25

In [ ]:
pd.read

In [25]:
active_datamodule.has_train_data(-2)

False

In [13]:
df["labelling_round"].value_counts()

-1    25
 4    25
 3    25
 0    25
 2    25
 1    25
Name: labelling_round, dtype: int64

In [ ]:
active_datamodule.set_labelled_dataset(df)

In [ ]:
active_datamodule._df[SpecialKeys.IS_LABELLED].value_counts()

In [ ]:
df[SpecialKeys.IS_LABELLED].value_counts()

In [ ]:
df2 = active_datamodule._df

In [ ]:
df2.columns

In [ ]:
df2[SpecialKeys.ID]

In [ ]:
s_cols = ["is_labelled", "is_validation", "labelling_round", "labels"]
cols = df2.columns[~df2.columns.isin(s_cols)]

to_update = pd.merge(df2[cols], df, on="unique_id", how="inner").head()
to_keep = df2.loc[~df2["unique_id"].isin(df["unique_id"])]

pd.concat([to_update, to_keep])

In [ ]:
with local_seed(42):
    model = AutoModelForSequenceClassification.from_pretrained(
        metadata["name_or_path"],
        id2label=datamodule.id2label,
        label2id=datamodule.label2id,
        num_labels=len(datamodule.labels),
    )

In [ ]:
list_dfs = []

for _ in range(3):
    active_datamodule = ClassificationActiveDataModule.from_dataset_dict(
        dataset_dict, tokenizer, seed=42, batch_size=64, eval_batch_size=256
    )

    active_estimator = RandomStrategyForSequenceClassification(
        model=deepcopy(model),
        seed=42,
        loggers=[TensorBoardLogger("logs")],
        deterministic=True,
    )

    active_estimator.active_fit(
        active_datamodule,
        max_rounds=2,
        # max_budget=200,
        # limit_train_batches=20,
        # limit_validation_batches=22,
        # limit_test_batches=21,
        # limit_pool_batches=30,
        # validation_perc=0.4,
        # validation_interval=1,
        # validation_sampling="stratified",
        query_size=25,
    )

    list_dfs.append(active_datamodule.get_labelled_dataset())
    # from copy import deepcopy
    # tb_logger2 = TensorBoardLogger("logs2")
    # active_estimator2 = RandomStrategyForSequenceClassification(
    #     model=deepcopy(model), seed=42, loggers=[tb_logger2],
    # )

In [ ]:
(
    list_dfs[1].equals(list_dfs[2]),
    list_dfs[1].equals(list_dfs[0]),
    list_dfs[0].equals(list_dfs[2]),
)

In [ ]:
active_estimator.active_fit(
    active_datamodule,
    max_rounds=2,
    # max_budget=200,
    limit_train_batches=20,
    limit_validation_batches=22,
    limit_test_batches=21,
    limit_pool_batches=30,
    validation_perc=0.4,
    validation_interval=1,
    validation_sampling="stratified",
    query_size=25,
)

In [ ]:
active_estimator.progress_tracker.budget_tracker

In [ ]:
active_estimator.progress_tracker.round_tracker

In [ ]:
active_datamodule.data_statistics()

In [ ]:
active_estimator2.replay_active_fit(
    active_datamodule,
    limit_train_batches=20,
    limit_validation_batches=22,
    limit_test_batches=21,
    limit_pool_batches=30,
)

In [ ]:
active_estimator.progress_tracker.budget_tracker

In [ ]:
active_datamodule.query_size

In [ ]:
active_datamodule.train_loader(0).dataset

In [ ]:
df = active_datamodule._df

In [ ]:
df.loc[(df["labelling_round"] < 0) & (df["is_labelled"] == True)].shape

In [ ]:
active_datamodule.has_train_data(0 - 1), active_datamodule.train_size(0 - 1)

In [ ]:
df.labelling_round.value_counts()

In [ ]:
active_estimator = UncertaintyBasedStrategyForSequenceClassification(
    model=model, score_fn="entropy"
)

In [ ]:
active_estimator.active_fit(
    active_datamodule,
    max_rounds=2,
    max_budget=36500,
    limit_train_batches=20,
    limit_validation_batches=22,
    limit_test_batches=21,
    limit_pool_batches=30,
    # val_perc=0.4,
    query_size=25,
)

In [ ]:
active_datamodule.data_statistics

In [ ]:
df = (
    dataset_dict["validation"]
    .to_pandas()
    .drop(columns=["input_ids", "attention_mask", "token_type_ids"])
    .assign(labels=lambda df_: df_["labels"].map(datamodule.id2label))
)

In [ ]:
df = pd.concat(
    [
        df,
        pd.DataFrame(
            data=softmax(validation_out["logits"], axis=-1),
            columns=datamodule.id2label.values(),
        ),
    ],
    axis=1,
)

In [ ]:
df["entropy"] = entr(df.iloc[:, -4:].values).sum(-1)
df["pred"] = df.iloc[:, -4:].values.argmax(-1)
df["pred"] = df["pred"].map(datamodule.id2label)

In [ ]:
validation_out["metrics"]

In [ ]:
validation_out["unique_id"]

In [ ]:
df = pd.DataFrame(
    data=validation_out["logits"],
    columns=list(range(validation_out["logits"].shape[1])),
).assign(
    pred=lambda df_: df_.values.argmax(-1),
    unique_id=validation_out["unique_id"],
)

In [ ]:
df = pd.merge(
    df, dataset_dict["validation"].to_pandas(), on="unique_id", how="inner"
)

In [ ]:
df = df.iloc[:, :-3]

In [ ]:
df = df.assign(
    pred=lambda df_: df_["pred"].map(datamodule.id2label),
    labels=lambda df_: df_["labels"].map(datamodule.id2label),
)

In [ ]:
(df["pred"] == df["labels"]).mean()

In [ ]:
df["entropy"] = entr(softmax(df.iloc[:, :4], axis=-1)).sum(-1)

In [ ]:
df.sort_values("entropy", ascending=False)